In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver # módulo que contiene implementaciones de controladores de navegadores web
from webdriver_manager.chrome import ChromeDriverManager # controlador de Chrome  
from selenium.webdriver.support import expected_conditions as EC # método para escribir códigos que esperan hasta que ciertas condiciones sean cumplidas
from selenium.webdriver.support.ui import WebDriverWait # método para escribir códigos que utilizan esperas implícitas o explícitas
from selenium.webdriver.common.by import By # método para localizar elementos por sus atributos HTML
from selenium.webdriver import ActionChains # módulo para implementar interacciones con el navegador from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#d = webdriver.Chrome('your-chrome-driver-path',chrome_options=chrome_options)

In [3]:
# Inicializa la página web en el navegador Chrome
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chrome_options)
url = 'https://www.despegar.com.co/'
driver.get(url) # aca le pido al webdriver que me traiga la pagina que le solicite en url

[WDM] - Current google-chrome version is 97.0.4692
[WDM] - Get LATEST driver version for 97.0.4692
[WDM] - Driver [/home/genesis/.wdm/drivers/chromedriver/linux64/97.0.4692.71/chromedriver] found in cache


In [4]:
elemento_oferta = '.shifu-3-header .header-corners-container .header-product-item .shifu-3-button-circle.OFFERS'
delay = 10
try:
    element_present = EC.presence_of_element_located((By.CSS_SELECTOR, elemento_oferta))
    WebDriverWait(driver, delay).until(element_present)
except TimeoutError:
    print('Timed out waiting for page to load')
print(driver.current_url)


https://www.despegar.com.co/


In [5]:
element1 = driver.find_element_by_css_selector(elemento_oferta)
ActionChains(driver).click(element1).perform()
print(driver.current_url)

https://www.despegar.com.co/ofertas-de-viajes


In [6]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()


In [7]:
etiqueta_promociones = soup.select('slider-item')  # selecciona todos los elementos que tienen la clase slider-item



In [8]:
def chequear_nulo(etiqueta):
    if etiqueta is not None:
        return etiqueta.text

def chequear_imagen(etiqueta):
    if etiqueta is not None:
        return etiqueta['src']

for etiqueta in (etiqueta_promociones):
    titulo = etiqueta.select_one('.offer-card-title.small-title ')
    imagen = etiqueta.select_one('.offer-card-image .offer-card-image-main')
    categoria = etiqueta.select_one('.offer-card-product')
    precio_oferta = etiqueta.select_one('.offer-card-pricebox-price-amount ')
    precio_normal = etiqueta.select_one('.offer-card-pricebox-price .offer-card-pricebox-price-old')
    print(chequear_nulo(titulo), chequear_imagen(imagen), chequear_nulo(categoria), chequear_nulo(precio_oferta), chequear_nulo(precio_normal))



Paquetes a Cartagena de Indias //media.staticontent.com/media/pictures/53d225ab-e0e8-4f60-b65b-29b43c71f07e/300x200 Paquete 604.883  $832.739 
Paquetes a Cali //media.staticontent.com/media/pictures/546c3a88-dac6-4fab-b473-338ae9654ee0/300x200 Paquete 386.943  $511.011 
Paquetes a San Andrés //media.staticontent.com/media/pictures/5cc13b62-c0dc-4143-b3e0-a895752c280e/300x200 Paquete 1.056.157  $1.387.889 
Paquetes a Cancún //media.staticontent.com/media/pictures/229e9dfd-402d-4908-bd75-692683d7f0fb/300x200 Paquete 2.394.168  $2.724.317 
Paquetes a Miami //media.staticontent.com/media/pictures/d2829cc9-46ae-42f8-ae68-d3ee165c7f4e/300x200 Paquete 2.637.483  $2.729.632 
Paquetes a Nueva York //media.staticontent.com/media/pictures/8e4ff942-b2fb-4740-9b17-9248fee71b7f/300x200 Paquete 2.272.543  $3.442.802 
Paquetes a Panamá //media.staticontent.com/media/pictures/ec13b227-2681-4918-8f94-efe96767c704/300x200 Paquete 1.070.001  $1.273.791 
Paquetes a Buenos Aires //media.staticontent.com/med